In [ ]:

                    ##############     Coding the Likelihood Function   ##############


import numpy as np
from scipy.stats import norm

def ar7_conditional_likelihood(params, data):
    """
    Calculate the conditional likelihood of an AR(7) model.

    Parameters:
        params (list): Parameters [phi1, phi2, ..., phi7, sigma^2].
        data (array): Array of observed data.

    Returns:
        float: Log likelihood value.
    """
    phi = params[:-1]
    sigma_squared = params[-1]
    n = len(data)
    ll = 0

    # First 7 observations are given, likelihood starts from the 8th observation
    for t in range(7, n):
        # Calculate mean based on past 7 observations
        mean = np.dot(phi, data[t-7:t][::-1])
        # Calculate log likelihood contribution of current observation
        ll += norm.logpdf(data[t], loc=mean, scale=np.sqrt(sigma_squared))

    return ll

def ar7_unconditional_likelihood(params, data):
    """
    Calculate the unconditional likelihood of an AR(7) model.

    Parameters:
        params (list): Parameters [phi1, phi2, ..., phi7, sigma^2].
        data (array): Array of observed data.

    Returns:
        float: Log likelihood value.
    """
    phi = params[:-1]
    sigma_squared = params[-1]
    n = len(data)
    ll = 0

    # Calculate the unconditional mean of the AR(7) process
    mu = np.dot(phi, data[:7][::-1])

    # Calculate log likelihood for each observation using unconditional mean
    for t in range(n):
        ll += norm.logpdf(data[t], loc=mu, scale=np.sqrt(sigma_squared))

    return ll

# Example usage:
# Define some sample data and initial parameters
data = np.random.normal(size=100)  # Sample data
initial_params = [0.5, -0.2, 0.3, -0.1, 0.2, -0.4, 0.1, 1.0]  # Initial parameters

# Calculate conditional likelihood
conditional_ll = ar7_conditional_likelihood(initial_params, data)
print("Conditional Log Likelihood:", conditional_ll)

# Calculate unconditional likelihood
unconditional_ll = ar7_unconditional_likelihood(initial_params, data)
print("Unconditional Log Likelihood:", unconditional_ll)


In [ ]:

                             #########      Maximizing the Likelihood  ############


from scipy.optimize import minimize

# Define negative log likelihood functions (to be minimized)
def neg_log_likelihood_conditional(params, data):
    return -ar7_conditional_likelihood(params, data)

def neg_log_likelihood_unconditional(params, data):
    return -ar7_unconditional_likelihood(params, data)

# Example usage:
# Define some sample data
data = np.random.normal(size=100)  # Sample data

# Initial parameter guess
initial_params = [0.5, -0.2, 0.3, -0.1, 0.2, -0.4, 0.1, 1.0]

# Maximize conditional likelihood
result_conditional = minimize(neg_log_likelihood_conditional, initial_params, args=(data,), method='Nelder-Mead')
print("Maximized Conditional Likelihood Parameters:", result_conditional.x)

# Maximize unconditional likelihood
result_unconditional = minimize(neg_log_likelihood_unconditional, initial_params, args=(data,), method='Nelder-Mead')
print("Maximized Unconditional Likelihood Parameters:", result_unconditional.x)


In [ ]:
                                 #########      Parameter Estimation   ############


import numpy as np
import pandas as pd
from scipy.optimize import minimize

# Step 1: Data Preprocessing
# Load FRED-MD dataset
data = pd.read_csv('/content/current.csv')  # Replace 'path_to_your_dataset.csv' with the actual path

# Assuming 'INDPRO' is the column name for the INDPRO variable
indpro = data['INDPRO']

# Calculate monthly log differences
log_diff_indpro = np.log(indpro).diff().dropna()

# Step 2: Likelihood Functions
def ar7_conditional_likelihood(params, data):
    phi, sigma_sq = params[:7], params[7]
    n = len(data)
    ll = 0
    for t in range(7, n):
        yt = data[t]
        mu_t = np.dot(phi, data[t-7:t][::-1])  # Reverse the order to align with AR(7) formula
        ll += -0.5 * (np.log(2 * np.pi * sigma_sq) + (yt - mu_t)**2 / sigma_sq)
    return ll

def ar7_unconditional_likelihood(params, data):
    phi, sigma_sq = params[:7], params[7]
    n = len(data)
    mu0 = np.mean(data[:7])  # Initial mean
    ll = -0.5 * (7 * np.log(2 * np.pi * sigma_sq) + np.sum((data[:7] - mu0)**2) / sigma_sq)
    for t in range(7, n):
        yt = data[t]
        mu_t = np.dot(phi, data[t-7:t][::-1])  # Reverse the order to align with AR(7) formula
        ll += -0.5 * (np.log(2 * np.pi * sigma_sq) + (yt - mu_t)**2 / sigma_sq)
    return ll

# Step 3: Parameter Estimation
# Initial parameter guess
initial_params = np.array([0.5, -0.2, 0.3, -0.1, 0.2, -0.4, 0.1, 1.0])

# Maximize conditional likelihood
result_conditional = minimize(ar7_conditional_likelihood, initial_params, args=(log_diff_indpro,), method='Nelder-Mead')
print("Maximized Conditional Likelihood Parameters:", result_conditional.x)

# Maximize unconditional likelihood
result_unconditional = minimize(ar7_unconditional_likelihood, initial_params, args=(log_diff_indpro,), method='Nelder-Mead')
print("Maximized Unconditional Likelihood Parameters:", result_unconditional.x)


In [ ]:
                                    #########      Forecasting   ############

import numpy as np

# Function to forecast future values
def forecast_ar7(params, data, steps):
    # Extract parameters
    phi = params[:7]
    sigma = params[7]

    # Initialize forecast array with observed data
    forecast = np.copy(data)

    # Forecast future values
    for i in range(steps):
        forecast_next = np.dot(phi, forecast[-7:]) + np.random.normal(0, sigma)
        forecast = np.append(forecast, forecast_next)

    return forecast

# Forecast using conditional likelihood parameters
conditional_forecast = forecast_ar7(result_conditional.x, log_diff_indpro, 8)
print("Conditional Forecast:", conditional_forecast[-8:])

# Forecast using unconditional likelihood parameters
unconditional_forecast = forecast_ar7(result_unconditional.x, log_diff_indpro, 8)
print("Unconditional Forecast:", unconditional_forecast[-8:])

     